In [ ]:
import import_ipynb
from Model3 import Lstm
from Dataset import c3_data

import os
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import optimizers
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
x_train, x_test, y_train, y_test, scaler_x, scaler_y = c3_data()

model = Lstm(output=5)

model.build(input_shape=[None, 2, 1])

model.summary()

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss=tf.losses.MeanSquaredError()
)


In [ ]:
model.load_weights('./model/model3_weight')

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=512, epochs=300, validation_data=(x_test, y_test), validation_batch_size=512)

model.save_weights('./model/model3_weight')


In [ ]:
predict_train = model(x_train)
predict_test = model(x_test)

r2_train = r2_score(y_train, predict_train)
mse_train = mean_squared_error(y_train, predict_train)
print('Training R2:', r2_train)
print('Training MSE:', mse_train)

r2_test = r2_score(y_test, predict_test)
mse_test = mean_squared_error(y_test, predict_test)
print('Testing R2:', r2_test)
print('Testing MSE:', mse_test)

y_train_true = scaler_y.inverse_transform(y_train)
y_train_pred = scaler_y.inverse_transform(predict_train)
y_test_true = scaler_y.inverse_transform(y_test)
y_test_pred = scaler_y.inverse_transform(predict_test)

In [ ]:
titles = ['Print temperature', 'Print speed', 'Print angle', 'Layer hight', 'Fill rate']
directory = './figure/c3/'
if not os.path.exists(directory):
    os.makedirs(directory)

fig, axs = plt.subplots(len(titles), 1, figsize=(10, 10))

for i in range(len(titles)):
    axs[i].plot(y_true[:150, i], '#6FA1D5', label='True')  
    axs[i].plot(y_pre[:150, i], 'r--', label='Predicted')  
    
    axs[i].set_xlabel('Sample Index')
    axs[i].set_ylabel(titles[i])
    
    y_max = max(max(y_true[:, i]), max(y_pre[:, i]))
    ylim_factors = [1.003, 1, 0.95, 1, 1]  
    if i == 2:
        axs[i].set_ylim(-15, 100)
    else:
        axs[i].set_ylim(top=y_max * ylim_factors[i])
    
    axs[i].text(0.99, 0.95, f'({chr(97 + i)})', transform=axs[i].transAxes, fontsize=12, fontweight='bold', va='top', ha='right')
    
lines, labels = axs[-1].get_legend_handles_labels()
fig.legend(lines, ['True', 'Predicted'], loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.05))

plt.tight_layout()

figure_path = os.path.join(directory, f"{titles[i]}.png")

plt.savefig(figure_path, dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
history = model.fit(
    x=x_train, 
    y=y_train, 
    batch_size=512, 
    epochs=300, 
    validation_data=(x_test, y_test), 
    validation_batch_size=512
)


In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(history.history['loss'], label='Training Loss', color='red', linewidth=2, linestyle='--')
plt.plot(history.history['val_loss'], label='Validation Loss', color='#6FA1D5', linewidth=2, linestyle='-')


# plt.title('Training and Validation Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)

plt.legend(loc='upper right', frameon=True, edgecolor='black', fontsize=14)

plt.grid(True, linestyle=':', color='grey', alpha=0.6)
plt.tick_params(axis='both', which='major', labelsize=14)

plt.text(0.02, 0.95, '(a)', transform=plt.gca().transAxes, fontsize=14, fontweight='bold', va='top')

plt.tight_layout()

plt.savefig('./figure/training_validation_loss_improved.JPEG', dpi=300)

plt.show()


In [ ]:
epochs_ranges = [(70, 100), (110, 140), (220, 250)]
directory = './figure/'
if not os.path.exists(directory):
    os.makedirs(directory)

fig, axs = plt.subplots(len(epochs_ranges), 1, figsize=(10, 8))

for idx, (start, end) in enumerate(epochs_ranges):
    axs[idx].plot(range(start, end), history.history['loss'][start:end], label='Training Loss', color='red', linewidth=2, linestyle='--')
    axs[idx].plot(range(start, end), history.history['val_loss'][start:end], label='Validation Loss', color='#6FA1D5', linewidth=2, linestyle='-')

    axs[idx].set_xlabel('Epochs', fontsize=14)
    axs[idx].set_ylabel('Loss', fontsize=14)

    axs[idx].grid(True, linestyle=':', color='grey', alpha=0.6)
    axs[idx].tick_params(axis='both', which='major', labelsize=14)
    axs[idx].text(0.02, 0.95, f'({chr(98 + idx)})', transform=axs[idx].transAxes, fontsize=14, fontweight='bold', va='top')

plt.tight_layout(rect=[0, 0.05, 1, 1])  

figure_path = os.path.join(directory, "training_validation_loss_combined.JPEG")
plt.savefig(figure_path, dpi=300)

plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10, 6))
colors = ['#005E57', '#009B8E', '#6FA1D5', '#00D4DB', 'orange']  

for i in range(y_pre.shape[1]):
    if i == 0:
        feature_name = "Print temperature"
    elif i == 1:
        feature_name = "Print speed"
    elif i == 2:
        feature_name = "Print angle"
    elif i == 3:
        feature_name = "Layer height"
    else:
        feature_name = "Fill rate"
    
    plt.scatter(y_true[:150, i], y_pre[:150, i], label=f'{feature_name}', color=colors[i])  


plt.xlabel('Actual Values', fontsize=14)
plt.ylabel('Predicted Values', fontsize=14)
plt.legend(fontsize=14)  

plt.plot([0, 230], [0, 230], color='red', linestyle='--', zorder=-1)
plt.grid(True)

plt.tight_layout()
plt.savefig('./figure/actual_vs_predicted.png', dpi=300)
plt.show()
